In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4


In [3]:
import openai
import os
import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install langchain
!pip install tiktoken
!pip install chromadb langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip show protobuf
!pip show tensorflow
!pip show transformers
!pip show tokenizers

Name: protobuf
Version: 5.29.2
Summary: 
Home-page: https://developers.google.com/protocol-buffers/
Author: protobuf@googlegroups.com
Author-email: protobuf@googlegroups.com
License: 3-Clause BSD License
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: google-ai-generativelanguage, google-api-core, google-cloud-aiplatform, google-cloud-bigquery-connection, google-cloud-bigquery-storage, google-cloud-bigtable, google-cloud-datastore, google-cloud-firestore, google-cloud-functions, google-cloud-iam, google-cloud-language, google-cloud-pubsub, google-cloud-resource-manager, google-cloud-translate, google-generativeai, googleapis-common-protos, grpc-google-iam-v1, grpcio-status, onnxruntime, opentelemetry-proto, orbax-checkpoint, proto-plus, tensorboard, tensorflow, tensorflow-datasets, tensorflow-hub, tensorflow-metadata, wandb
Name: tensorflow
Version: 2.17.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.ten

In [14]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [16]:
import requests
from bs4 import BeautifulSoup

In [18]:
def extract_content_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    content = " ".join([p.get_text().strip() for p in paragraphs])
    return content

In [19]:
url = "https://en.wikipedia.org/wiki/Pakistan_Super_League"
clean_text = extract_content_from_url(url)

print(f"Extracted Text Length: {len(clean_text)}")
print("Sample Extracted Text (First 500 characters):\n", clean_text[:500])

Extracted Text Length: 11943
Sample Extracted Text (First 500 characters):
  The Pakistan Super League, also known as PSL and HBL PSL for sponsorship reasons, is a men's T20 cricket league held annually in Pakistan. Founded by the PCB in 2015, the league features six city-based franchise teams. Instead of operating as an association of independently owned teams, the league is a single entity in which each franchise is owned and controlled by the investors.[3] Each team plays the group stage matches in a  double round robin format and the top four teams with the most poi


In [20]:
from langchain.text_splitter import CharacterTextSplitter

In [21]:
def split_text_into_chunks(text, chunk_size=500, chunk_overlap=50):
    text_splitter = CharacterTextSplitter(
        separator=" ",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = text_splitter.split_text(text)
    output_dir = "output_multiple"
    os.makedirs(output_dir, exist_ok=True)
    for i, chunk in enumerate(chunks):
        file_path = os.path.join(output_dir, f"chunk_{i + 1}.json")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(chunk)
    return chunks

In [22]:
chunks = split_text_into_chunks(clean_text)

In [23]:
chunks

["The Pakistan Super League, also known as PSL and HBL PSL for sponsorship reasons, is a men's T20 cricket league held annually in Pakistan. Founded by the PCB in 2015, the league features six city-based franchise teams. Instead of operating as an association of independently owned teams, the league is a single entity in which each franchise is owned and controlled by the investors.[3] Each team plays the group stage matches in a double round robin format and the top four teams with the most",
 'robin format and the top four teams with the most points qualify for the playoffs, culminating in the final. There have been nine seasons of the league. Islamabad United are the most successful team, winning the title three times. Lahore Qalandars are the only team to successfully defend its title, after winning in the 2023 season. In September 2015, the Pakistan Cricket Board officially announced the launch of the PSL. Former Pakistani national team captains Wasim Akram and Rameez Raja signed'

In [25]:
!pip install langchain openai

In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [29]:
def store_embeddings_with_chroma(chunks, api_key, persist_directory="./chroma_db"):
    embeddings_model = OpenAIEmbeddings(api_key=api_key)

    vector_store = Chroma(
        collection_name="document_chunks",
        embedding_function=embeddings_model,
        persist_directory=persist_directory
    )

    vector_store.add_texts(chunks)
    vector_store.persist()

    return vector_store

In [30]:
vector_store = store_embeddings_with_chroma(chunks, api_key)
print(f"Stored {vector_store._collection.count()} chunks in ChromaDB.")

<ipython-input-29-642d1f252418>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings_model = OpenAIEmbeddings(api_key=api_key)
<ipython-input-29-642d1f252418>:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


Stored 27 chunks in ChromaDB.


<ipython-input-29-642d1f252418>:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist()


In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [32]:
def get_conversation_chain(vector_store):
    llm = ChatOpenAI(api_key=api_key)
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vector_store.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [33]:
def respond_to_query(query, conversation_chain):
    full_response = conversation_chain({"question": query})
    return full_response['answer']

In [36]:
vectorstore = store_embeddings_with_chroma(chunks, api_key)
conversation_chain = get_conversation_chain(vectorstore)
user_query = input()
response = respond_to_query(user_query, conversation_chain)
print(response)

when was the psl established, and by whom
The Pakistan Super League (PSL) was established by the Pakistan Cricket Board (PCB) in 2015.
